In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [ ]:
url = "https://www.realtor.com/realestateandhomes-search/Houston_TX"
response = requests.get(url)
if response.status_code == 200:
    print(f"Successfully fetched data from {url}.")
    print("Response Content:")
elif response.status_code == 403:
    print(f"Access forbidden for {url}.")
elif response.status_code == 404:
    print(f"Page not found for {url}.")
else:
    print(f"Failed to fetch data from {url}. Status code: {response.status_code}")

Access forbidden for https://www.realtor.com/realestateandhomes-search/Houston_TX.


In [ ]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36"
}
def fetch_listings(url):
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
      soup = BeautifulSoup(response.content, 'html.parser')
      property_listings = soup.find_all('div', class_='BasePropertyCard_propertyCard__JXqlE')

      data = {
            'Price': [],
            'Address': [],
            'URL': [],
            'Lot Size': []
        }
      for listing in property_listings:
          # Extract price
          price = listing.find('div', class_='Price__Component-rui__x3geed-0').text.strip()

          # Extract address
          address = listing.find('div', class_='truncate-line')
          address_text = ', '.join([line.strip() for line in address.stripped_strings])

          # Extract size of the lot
          lot_size_element = listing.find('li', class_='PropertyLotSizeMetastyles__StyledPropertyLotSizeMeta-rui__sc-1cz4zco-0')
          lot_size = lot_size_element.text.strip() if lot_size_element else 'N/A'
          # Append data to the dictionary
          data['Price'].append(price)
          data['Address'].append(address_text)
          data['URL'].append(url)
          data['Lot Size'].append(lot_size)
      # Create a Pandas DataFrame
      df = pd.DataFrame(data)
      print(df.head(50))
      # Save the DataFrame to an Excel file
      df.to_excel('property_listings.xlsx', index=False)

    else:
      print(f"having some problems, status code: {response.status_code}, maybe you can check the user-agent.")
    return df
fetch_listings(url)

          Price                                            Address  \
0      $479,500             8669 Green Kolbe Ln, Houston, TX 77080   
1      $289,900             12306 May Laurel Dr, Houston, TX 77014   
2      $340,000             5619 Spanish Oak Dr, Houston, TX 77066   
3      $375,000               8206 W Tidwell Rd, Houston, TX 77040   
4      $198,500        351 Lakeside Ln Apt 104, Seabrook, TX 77058   
5      $388,888                6023 McKnight St, Houston, TX 77035   
6   $65,000,000              107 Timberwilde Ln, Houston, TX 77024   
7      $159,900               4806 Airport Blvd, Houston, TX 77048   
8      $449,950  Frances Plan, Knoll Court Community, Houston, ...   
9       $68,000        5625 Antoine Dr Apt 1018, Houston, TX 77091   
10     $316,990  Rochester Plan, Sandrock Station Community, Ho...   
11     $309,990  Lancaster II Plan, Sandrock Station Community,...   
12     $269,990  Southport II Plan, Sandrock Station Community,...   
13     $331,990  Wil

,Price,Address,URL,Lot Size
0,"$479,500","8669 Green Kolbe Ln, Houston, TX 77080",https://www.realtor.com/realestateandhomes-sea...,"2,104sqft lot2,104 square foot lot"
1,"$289,900","12306 May Laurel Dr, Houston, TX 77014",https://www.realtor.com/realestateandhomes-sea...,0.25acre lot0.25 acre lot
2,"$340,000","5619 Spanish Oak Dr, Houston, TX 77066",https://www.realtor.com/realestateandhomes-sea...,"9,439sqft lot9,439 square foot lot"
3,"$375,000","8206 W Tidwell Rd, Houston, TX 77040",https://www.realtor.com/realestateandhomes-sea...,0.32acre lot0.32 acre lot
4,"$198,500","351 Lakeside Ln Apt 104, Seabrook, TX 77058",https://www.realtor.com/realestateandhomes-sea...,4.71acre lot4.71 acre lot
...,...,...,...,...
57,"$274,900","10307 Den Oak Dr, Houston, TX 77065",https://www.realtor.com/realestateandhomes-sea...,N/A
58,"$463,000","12014 Quiet Water Ct, Houston, TX 77065",https://www.realtor.com/realestateandhomes-sea...,N/A
59,"$405,000","159 Castlegate Ln, Houston, TX 77065",https://www.realtor.com/realestateandhomes-sea...,N/A
60,"$330,000","11823 Berkway Trl, Houston, TX 77065",https://www.realtor.com/realestateandhomes-sea...,N/A


In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import random

# Lista de User-Agents para simular diferentes navegadores
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36",
    # Agrega más User-Agents si lo deseas
]

base_url = "https://www.realtor.com/realestateandhomes-search/{}/type-land"

def fetch_listings_for_state(state):
    state_data = {
        'Price': [],
        'Address': [],
        'URL': [],
        'Lot Size': []
    }

    # Start with the first page
    page_number = 1
    has_more_pages = True

    while has_more_pages:
        # Elegir un User-Agent aleatorio de la lista
        user_agent = random.choice(user_agents)
        headers = {
            "User-Agent": user_agent
        }

        # Agregar una pausa aleatoria entre 5 y 15 segundos
        time.sleep(0.5)

        url = base_url.format(state) + f"/pg-{page_number}"
        print(url)
        response = requests.get(url, headers=headers)
        print(page_number)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            property_listings = soup.find_all('div', class_='BasePropertyCard_propertyCard__JXqlE')

            if not property_listings:
                # No more pages with listings
                has_more_pages = False
                break

            for listing in property_listings:
                # Extract price
                price = listing.find('div', class_='Price__Component-rui__x3geed-0').text.strip()

                # Extract address
                address_div = listing.find('div', class_='card-address truncate-line')
                address_elements = address_div.find_all('div', class_='truncate-line')
                address_text = ', '.join([element.text.strip() for element in address_elements])

                # Extract URL of the property listing
                anchor = listing.find('a', class_='LinkComponent_anchor__2uAhr')
                url = anchor['href'] if anchor and 'href' in anchor.attrs else ''

                # Extract size of the lot
                lot_size_element = listing.find('li', class_='PropertyLotSizeMetastyles__StyledPropertyLotSizeMeta-rui__sc-1cz4zco-0')
                lot_size = lot_size_element.text.strip() if lot_size_element else 'N/A'

                # Append data to the dictionary
                state_data['Price'].append(price)
                state_data['Address'].append(address_text)
                state_data['URL'].append(url)
                state_data['Lot Size'].append(lot_size)

            # Move to the next page
            page_number += 1
        elif response.status_code == 404:
            print(f"No hay más páginas para {state}")
            break
        else:
            print(f"Error en la solicitud para {state}")
            print(response.status_code)
            break

    # Create a Pandas DataFrame for the state's data
    df = pd.DataFrame(state_data)

    # Save the DataFrame to an Excel file
    filename = f"property_listings_{state}.xlsx"
    df.to_excel(filename, index=False)

# Lista de 50 estados en los Estados Unidos
states = ['Washington', 'West-Virginia', 'Wisconsin']

# Loop through each state and fetch the property listings
for state in states:
    fetch_listings_for_state(state)


https://www.realtor.com/realestateandhomes-search/Washington/type-land/pg-1
1
https://www.realtor.com/realestateandhomes-search/Washington/type-land/pg-2
2
https://www.realtor.com/realestateandhomes-search/Washington/type-land/pg-3
3
https://www.realtor.com/realestateandhomes-search/Washington/type-land/pg-4
4
https://www.realtor.com/realestateandhomes-search/Washington/type-land/pg-5
5
https://www.realtor.com/realestateandhomes-search/Washington/type-land/pg-6
6
https://www.realtor.com/realestateandhomes-search/Washington/type-land/pg-7
7
https://www.realtor.com/realestateandhomes-search/Washington/type-land/pg-8
8
https://www.realtor.com/realestateandhomes-search/Washington/type-land/pg-9
9
https://www.realtor.com/realestateandhomes-search/Washington/type-land/pg-10
10
https://www.realtor.com/realestateandhomes-search/Washington/type-land/pg-11
11
https://www.realtor.com/realestateandhomes-search/Washington/type-land/pg-12
12
https://www.realtor.com/realestateandhomes-search/Washingt

In [ ]:
!pip install chromedriver_autoinstaller

REdfin


In [ ]:
import os
import random
import time
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.proxy import Proxy, ProxyType
import chromedriver_autoinstaller

# Especifica la ruta de Chrome manualmente (modifica esto con tu propia ruta si es diferente)
chrome_path = "C:/Program Files/Google/Chrome/Application/chrome.exe"

# URL principal
mainpart_url = "https://www.redfin.com"
url_principal = "https://www.redfin.com/state/{estado}"
filter = "/filter/property-type=land"

# Lista de estados de los EE. UU.
estados = ["Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", "Delaware", "Florida", "Georgia", "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana", "Maine", "Maryland", "Massachusetts", "Michigan", "Minnesota", "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada", "New Hampshire", "New Jersey", "New Mexico", "New York", "North Carolina", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Vermont", "Virginia", "Washington", "West Virginia", "Wisconsin", "Wyoming"]

def obtener_enlaces_ciudades(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    tabla = soup.find("table", class_="filterableTable")
    if tabla is not None:
        enlaces = tabla.find_all("a", target="_blank")
        return enlaces
    else:
        print(f"No se encontró la tabla en {url}")
        return []

# Lista de proxies HTTP gratuitos (actualiza con proxies fiables y funcionales si los conoces)
proxies = [
    "http://145.239.85.58:9300",
    "socks5://46.4.96.137:1080",
    "socks5://47.91.88.100:1080",
    "http://45.77.56.114:30205",
    "socks5://82.196.11.105:1080",
    "https://51.254.69.243:3128",
    "socks5://178.62.193.19:1080",
    "socks5://188.226.141.127:1080",
    "socks5://217.23.6.40:1080",
    "http://185.153.198.226:32498",
    "https://81.171.24.199:3128",
    "socks5://5.189.224.84:10000",
    "socks5://108.61.175.7:31802",
    "https://176.31.200.104:3128",
]

# Configuración de Selenium
chromedriver_autoinstaller.install()  # Instalar la versión adecuada de chromedriver automáticamente
options = Options()
options.add_argument('--headless')  # Ejecutar el navegador en modo headless (sin interfaz gráfica)
options.add_argument('--no-sandbox')  # Evitar problemas en entornos sin pantalla
options.add_argument('--disable-dev-shm-usage')  # Evitar problemas en entornos sin pantalla

# Establecer la ruta de Chrome manualmente
options.binary_location = chrome_path

# Crear un directorio para guardar los archivos descargados
if not os.path.exists("archivos_descargados"):
    os.makedirs("archivos_descargados")

# Procesar cada estado
for estado in estados:
    url_estado = url_principal.format(estado=estado)
    enlaces_ciudades = obtener_enlaces_ciudades(url_estado)
    for enlace in enlaces_ciudades:
        format_url = enlace["href"]
        url_city = mainpart_url + format_url + filter

        # Intentar cada proxy hasta encontrar uno que funcione
        for proxy in proxies:
            prox = Proxy()
            prox.proxy_type = ProxyType.MANUAL
            prox.http_proxy = proxy
            prox.ssl_proxy = proxy

            capabilities = webdriver.DesiredCapabilities.CHROME
            prox.add_to_capabilities(capabilities)

            driver = webdriver.Chrome(service=Service('./chromedriver'), options=options, desired_capabilities=capabilities)

            try:
                driver.get(url_city)
                time.sleep(5)  # Pausa para permitir que la página se cargue completamente

                # Encontrar el enlace "Download All" y hacer clic en él
                download_link = driver.find_element(By.CLASS_NAME, 'downloadLink')
                download_url = download_link.get_attribute('href')
                driver.get(download_url)

                # Retardo para asegurarse de que la descarga se complete
                time.sleep(5)

                # Cerrar la ventana de descarga
                driver.find_element(By.CSS_SELECTOR, 'body').send_keys(Keys.CONTROL + 'w')

                soup = BeautifulSoup(driver.page_source, "html.parser")
                title = soup.find("h1", class_="collapsedListView").text
                nombre_archivo = f"archivos_descargados/{title}.csv"

                # Mover el archivo descargado a la carpeta correspondiente
                os.rename('archivo_descargado.csv', nombre_archivo)

                print(f"El archivo {nombre_archivo} ha sido descargado exitosamente.")
                break  # Salir del bucle de proxies si la descarga fue exitosa
            except Exception as e:
                print(f"Error al obtener la página {url_city} con el proxy {proxy}. Error: {e}")
                continue  # Probar el siguiente proxy en caso de error

            driver.quit()  # Cerrar el navegador para liberar recursos

        # Retardo de 1-2 segundos entre las solicitudes para evitar sobrecargar el servidor
        time.sleep(1 + 1 * random.random())

# Cerrar el navegador al finalizar
driver.quit()


ValueError: ignored